In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [27]:
def get_Titre(soup):
    try:
        # Outer Tag Object
        titre = new_soup.find("span", attrs={'id': 'productTitle'})
        
        # Inner NavigatableString Object
        titre_value = titre.text

        # Title as a string value
        titre_string = titre_value.strip()

    except AttributeError:
        titre_string = ""

    return titre_string

def get_Rating(soup):
    try:
        # This typically gets the number of reviews, e.g., "1,234 ratings"
        rating = new_soup.find("span", attrs={'id':"acrCustomerReviewText",'class':"a-size-base"}).text.strip()
        
    except AttributeError:
        rating = ""	

    return rating

def get_Note(soup):
    try:
        # This usually gets the average star rating, e.g., "4.5 out of 5 stars"
        note = new_soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big'}).text
        
    except AttributeError:
        note = ""	

    return note

def get_Review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

def get_Disponibilité(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

def get_top_review(soup):
    try:
        review_div = soup.find("div", attrs={
            "class": "a-expander-content reviewText review-text-content a-expander-partial-collapse-content"
        })
        if review_div:
            # Extraire le texte du <span>, supprimer les <br>, et nettoyer les espaces
            review_text = review_div.get_text(separator=" ", strip=True)
            return review_text
        return None
    except Exception as e:
        return None




In [31]:
if __name__ == '__main__':
    
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})

    URL = 'https://www.amazon.com/s?k=playstation+5+controller&crid=AKB9WITXUDBH&sprefix=playsta%2Caps%2C292&ref=nb_sb_ss_p13n-pd-dpltr-ranker_4_7=AKB9WITXUDBH&dib=eyJ2IjoiMSJ9.15JJFCDWvpRvWSqT7PHtnsj95wkU_6bhD2LkAgVtK8LaGCkAXifdcuI2HOvK-eyp3S2_LkeOCKi6aIV743IA4XEgLN_zT5AVS6MoKZXGWrh2Ap1bDrb6eVl1RO_MgHZ1ykQTAaTQviok12lTtYxAZHgeVA_EVfq_Tply18j6JJubkMvTFbMDhmKp5_TvAptGCJ1Oucaj4nx8478ZUxWEZ5XqFcWPBRPAi2NRaNhj9fw.salfyx7FtQOGKHSIN0rtTPeSp5_1RhzHjXNnFM0BqDs&dib_tag=se&keywords=playstation+5+controller&qid=1752024181&sprefix=playsta%2Caps%2C292&sr=8-1'
    
    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, 'html.parser')

    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    links_list = []

    for link in links:
        # Get the full href attribute. It seems these links are already absolute.
        full_link = link.get('href')
        
        # Some Amazon links might be relative (e.g., starting with /dp/), 
        # while others might be absolute (e.g., starting with https://).
        # Based on your error, the 'link' is already absolute.
        # If sometimes it's relative, you'd need a check:
        # if not full_link.startswith('http'):
        #     full_link = "https://www.amazon.com" + full_link
            
        links_list.append(full_link)

    d = {"titre":[], "rating":[], "note":[], "review_count":[], "disponibilite":[], "review_div":[]}
    
    for link in links_list:    
         
    # Si le lien est relatif, on ajoute le domaine Amazon
        if not link.startswith("http"):
            full_link = "https://www.amazon.com" + link
        else:
            full_link = link
            
        new_webpage = requests.get(full_link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        d['titre'].append(get_Titre(new_soup))
        d['rating'].append(get_Rating(new_soup))
        d['note'].append(get_Note(new_soup))
        d['review_count'].append(get_Review_count(new_soup))
        d['review_div'].append(get_top_review(new_soup))
        d['disponibilite'].append(get_Disponibilité(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['titre'] = amazon_df['titre'].replace('', np.nan)
    amazon_df = amazon_df.dropna(subset=['titre'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [33]:
amazon_df

,titre,rating,note,review_count,disponibilite,review_div
0,PlayStation DualSense™ Wireless Controller – C...,"36,190 ratings",4.6 out of 5 stars,"36,190 ratings",In Stock,I got the PlayStation DualSense® Wireless Cont...
1,PlayStation DualSense Wireless Controller – Co...,"11,526 ratings",4.6 out of 5 stars,"11,526 ratings",,"Es una excelente compra, se siente bien a la m..."
2,Set for PS5 L1 R1 L2 R2 Trigger Buttons Replac...,2 ratings,,2 ratings,Only 13 left in stock - order soon.,No instructions included. A quick trip to YouT...
3,"Gamrombo LED Wireless Controller for PS5, Comp...",391 ratings,4.5 out of 5 stars,391 ratings,In Stock,I started playing with this controller on my P...
4,"Wired Controller for PS5 Controller, Gaming Co...",127 ratings,,127 ratings,In Stock,I really like this product. Use it almost ever...
5,Dinosoo RGB Wireless Controller for PS5/PlaySt...,80 ratings,4.4 out of 5 stars,80 ratings,In Stock,"The RGB lights are a cool touch, with differen..."
6,"Nonbliep LED for PS5 Controller, for PS5 Slim/...",76 ratings,4.5 out of 5 stars,76 ratings,In Stock,Awesome controller — looks amazing and works g...
7,TiMOVO PS5 Controller Keyboard with Green Back...,"1,025 ratings",,"1,025 ratings",In Stock,Keyboard responsiveness: Keyboard works really...
8,NACON Revolution 5 Pro Officially Licensed Pla...,421 ratings,,421 ratings,In Stock,Llego en tiempo y forma. Excelente producto.
9,NACON Revolution 5 Pro Officially Licensed Pla...,421 ratings,,421 ratings,In Stock,Llego en tiempo y forma. Excelente producto.
